In [1]:
# LSTM for sequence classification
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Convolution1D, Flatten, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


Using TensorFlow backend.


In [2]:
from google.colab import files 
uploaded = files.upload() 

Saving threshold4_balanced_training_set.csv to threshold4_balanced_training_set.csv


In [3]:
df = pd.read_csv('threshold4_balanced_training_set.csv', usecols=['CONTENT','sentiment'])
df.head(10)
df['label']=df['sentiment']
df['label'].value_counts()

neutral     1269
positive    1269
negative    1269
Name: label, dtype: int64

In [4]:
for i, item in enumerate (df['label']):
  if item=='positive':
    df['label'][i]=2
  if item=='neutral':
    df['label'][i]=1
  if item=='negative':
    df['label'][i]=0
df['label'].value_counts()

2    1269
1    1269
0    1269
Name: label, dtype: int64

In [5]:
from google.colab import files 
uploaded = files.upload() 

Saving 10news_11132019_mannual_labelled.csv to 10news_11132019_mannual_labelled.csv


In [0]:
tdf = pd.read_csv('10news_11132019_mannual_labelled.csv')
len(tdf)
x = df['CONTENT']
y = df['label']
x_test=tdf['CONTENT']
y_test=tdf['label']

In [16]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.2, random_state=SEED)
print (y_train)

3390    0
1297    2
3320    0
3466    0
1987    2
       ..
1659    2
1590    2
1230    1
840     1
2998    0
Name: label, Length: 3045, dtype: object


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=40000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)
sequences_val=tokenizer.texts_to_sequences(x_validation)
sequences_test=tokenizer.texts_to_sequences(x_test)

In [0]:

# Pad the sequence to the same length
max_review_length = 15000
x_train = pad_sequences(sequences, maxlen=max_review_length)
x_validation = pad_sequences(sequences_val, maxlen=max_review_length)
x_test = pad_sequences(sequences_test, maxlen=max_review_length)



In [12]:
# Using embedding from Keras
# from keras.utils import to_categorical
# y_binary = to_categorical(y_int)
embedding_vecor_length = 300
top_words = 40000
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))

# Convolutional model (3x conv, flatten, 2x dense)
model.add(Convolution1D(64, 3, padding='same'))
model.add(Convolution1D(32, 3, padding='same'))
model.add(Convolution1D(16, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(180,activation='softmax'))
model.add(Dropout(0.2))
model.add(Dense(3,activation='softmax'))

# Log to tensorboard
tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train.astype('int'), epochs=3, callbacks=[tensorBoardCallback], batch_size=64)

# Evaluation on the test set
scores = model.evaluate(x_validation, y_validation.astype('int'), verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3
3045/3045 [==============================] - 579s 190ms/step - loss: 1.1006 - acc: 0.3300

Epoch 2/3
3045/3045 [==============================] - 579s 190ms/step - loss: 1.0994 - acc: 0.3307
Epoch 3/3
3045/3045 [==============================] - 578s 190ms/step - loss: 1.0992 - acc: 0.3251
Accuracy: 32.41%


In [19]:
scores = model.evaluate(x_test, y_test.astype('int'), verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.10000000149011612


In [20]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
vocabulary_size=40000
max_words=15000
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 15000, 32)         1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 1,333,503
Trainable params: 1,333,503
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.compile(loss='sparse_categorical_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [22]:
batch_size = 64
num_epochs = 5

model.fit(x_train, y_train.astype('int'), batch_size=batch_size, epochs=num_epochs)

Epoch 1/5
3045/3045 [==============================] - 984s 323ms/step - loss: 1.0955 - acc: 0.3530
Epoch 2/5
3045/3045 [==============================] - 979s 322ms/step - loss: 1.0233 - acc: 0.5218
Epoch 3/5
3045/3045 [==============================] - 981s 322ms/step - loss: 0.8050 - acc: 0.6529
Epoch 4/5
3045/3045 [==============================] - 984s 323ms/step - loss: 0.5957 - acc: 0.7567
Epoch 5/5
3045/3045 [==============================] - 984s 323ms/step - loss: 0.4719 - acc: 0.8069


In [23]:
scores = model.evaluate(x_validation, y_validation.astype('int'), verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.4514435704924616


In [24]:
scores = model.evaluate(x_test, y_test.astype('int'), verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.6000000238418579
